In [21]:
# import nltk

# nltk.download('punkt')  # 下载 punkt 分词器
# nltk.download('punkt_tab')  # 下载 punkt_tab 分词器数据
# nltk.download('averaged_perceptron_tagger')  # 下载词性标注器
# nltk.download('averaged_perceptron_tagger_eng')  # 下载英文词性标注器


In [22]:
from langchain.document_loaders import DirectoryLoader

# 定义文件所在的路径
DOC_PATH = "./AI-Guide-and-Demos-zh_CN/Guide"

# 使用 DirectoryLoader 从指定路径加载文件。"*.md" 表示加载所有 .md 格式的文件，这里仅导入文章 10（避免当前文章的演示内容对结果的影响）
loader = DirectoryLoader(DOC_PATH, glob="10*.md")

# 加载目录中的指定的 .md 文件并将其转换为文档对象列表
documents = loader.load()

# 打印查看加载的文档内容
print(documents[0].page_content[:200])

什么是 Top-K 和 Top-P 采样？Temperature 如何影响生成结果？

在文章 09 中我们探讨了 Beam Search 和 Greedy Search，现在来聊聊 model.generate() 中常见的三个参数: top-k, top-p 和 temperature。

代码文件下载

在线链接：Kaggle | Colab

目录

采样方法概述

Top-K 采样详解



In [23]:
text = """长隆广州世界嘉年华系列活动的长隆欢乐世界潮牌玩圣节隆重登场，在揭幕的第一天就吸引了大批年轻人前往打卡。据悉，这是长隆欢乐世界重金引进来自欧洲的12种巨型花车重磅出巡，让人宛若进入五彩缤纷的巨人国；全新的超级演艺广场每晚开启狂热的电音趴，将整个狂欢氛围推向高点。

记者在现场看到，明日之城、异次元界、南瓜欢乐小镇、暗黑城、魔域，五大风格迥异的“鬼”域在夜晚正式开启，全新重磅升级的十大“鬼”屋恭候着各位的到来，各式各样的“鬼”开始神出“鬼”没：明日之城中丧尸成群出行，寻找新鲜的“血肉”。异次元界异形生物游走，美丽冷艳之下暗藏危机。暗黑城亡灵出没，诅咒降临。魔域异“鬼”横行，上演“血腥恐怖”。南瓜欢乐小镇小丑当家，滑稽温馨带来欢笑。五大“鬼”域以灯光音效科技情景+氛围营造360°沉浸式异域次元界探险模式为前来狂欢的“鬼”友们献上“惊奇、恐怖、搞怪、欢乐”的玩圣体验。持续23天的长隆欢乐玩圣节将挑战游客的认知极限，让你大开眼界！
据介绍，今年长隆玩圣节与以往相比更为隆重，沉浸式场景营造惊悚氛围，两大新“鬼”王隆重登场，盛大的“鬼”王出巡仪式、数十种集声光乐和高科技于一体的街头表演、死亡巴士酷跑、南瓜欢乐小镇欢乐电音、暗黑城黑暗朋克、魔术舞台双煞魔舞、异形魔幻等一系列精彩节目无不让人拍手称奇、惊叹不止的“玩圣”盛宴让 “鬼”友们身临其境，过足“戏”瘾！
"""
print(len(text))

581


In [24]:
# 相比简单按固定长度切割，它能更好地在自然分隔符（如段落、句子）处断开，减少无意义的截断。
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=100,   # 每个文本块的最大长度
    chunk_overlap=20  # 文本块之间的字符重叠数量
)
texts=text_splitter.split_text(text)
print(texts)

['长隆广州世界嘉年华系列活动的长隆欢乐世界潮牌玩圣节隆重登场，在揭幕的第一天就吸引了大批年轻人前往打卡。据悉，这是长隆欢乐世界重金引进来自欧洲的12种巨型花车重磅出巡，让人宛若进入五彩缤纷的巨人国；全新', '出巡，让人宛若进入五彩缤纷的巨人国；全新的超级演艺广场每晚开启狂热的电音趴，将整个狂欢氛围推向高点。', '记者在现场看到，明日之城、异次元界、南瓜欢乐小镇、暗黑城、魔域，五大风格迥异的“鬼”域在夜晚正式开启，全新重磅升级的十大“鬼”屋恭候着各位的到来，各式各样的“鬼”开始神出“鬼”没：明日之城中丧尸成群', '鬼”开始神出“鬼”没：明日之城中丧尸成群出行，寻找新鲜的“血肉”。异次元界异形生物游走，美丽冷艳之下暗藏危机。暗黑城亡灵出没，诅咒降临。魔域异“鬼”横行，上演“血腥恐怖”。南瓜欢乐小镇小丑当家，滑稽温', '血腥恐怖”。南瓜欢乐小镇小丑当家，滑稽温馨带来欢笑。五大“鬼”域以灯光音效科技情景+氛围营造360°沉浸式异域次元界探险模式为前来狂欢的“鬼”友们献上“惊奇、恐怖、搞怪、欢乐”的玩圣体验。持续23天的', '怖、搞怪、欢乐”的玩圣体验。持续23天的长隆欢乐玩圣节将挑战游客的认知极限，让你大开眼界！', '据介绍，今年长隆玩圣节与以往相比更为隆重，沉浸式场景营造惊悚氛围，两大新“鬼”王隆重登场，盛大的“鬼”王出巡仪式、数十种集声光乐和高科技于一体的街头表演、死亡巴士酷跑、南瓜欢乐小镇欢乐电音、暗黑城黑', '巴士酷跑、南瓜欢乐小镇欢乐电音、暗黑城黑暗朋克、魔术舞台双煞魔舞、异形魔幻等一系列精彩节目无不让人拍手称奇、惊叹不止的“玩圣”盛宴让', '“鬼”友们身临其境，过足“戏”瘾！']


In [27]:
# 加载编码模型
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "chuxin-llm/Chuxin-Embedding"
embedding_model = HuggingFaceEmbeddings(model_name=model_name)
print(embedding_model)
query_embedding = embedding_model.embed_query('如何更换花呗绑定银行卡')
print(query_embedding)

# 打印生成的嵌入向量的长度，向量长度应与模型的输出维度一致（这里是 1024），你也可以选择打印向量看看
print(f"嵌入向量的维度为: {len(query_embedding)}")

model_name='chuxin-llm/Chuxin-Embedding' cache_folder=None model_kwargs={} encode_kwargs={} query_encode_kwargs={} multi_process=False show_progress=False
[-0.01667790673673153, -0.018957387655973434, -0.024541446939110756, -0.005326236132532358, -0.026209957897663116, 0.016999023035168648, 0.045709479600191116, -0.00791156105697155, 0.014640122652053833, -0.04644867032766342, 0.028066139668226242, 0.011024595238268375, 0.04017814248800278, -0.006136871874332428, -0.008642987348139286, -0.03564732149243355, 0.02520645596086979, 0.01779654435813427, 0.0188994612544775, 0.017025338485836983, -0.03114274889230728, -0.038812074810266495, 0.03958439081907272, -0.00931332353502512, -0.02281337045133114, 0.025712627917528152, -0.007711008656769991, -0.003091663820669055, 0.043370913714170456, -0.04140067100524902, -0.002238829154521227, -0.022930776700377464, -0.0025858297012746334, -0.016320601105690002, -0.011686638928949833, -0.009339399635791779, -0.019400540739297867, 0.00587067333981394

In [28]:
from langchain.vectorstores import FAISS
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 每个文本块的最大长度
    chunk_overlap=100,  # 文本块之间的字符重叠数量
    #length_function=len,  # 可以省略
    #separators=["\n\n", "\n", " ", "。", ""]  # 可以省略
)
docs=text_splitter.split_documents(documents)
vectorstore = FAISS.from_documents(docs, embedding=embedding_model)
print(len(docs))

18


In [29]:
vectorstore.save_local("faiss_index")
# 加载向量数据库
vectorstore = FAISS.load_local("faiss_index",embeddings=embedding_model,allow_dangerous_deserialization=True)

In [30]:
# 创建检索器
retriever=vectorstore.as_retriever(search_kwargs={'k':3})
# k=3 表示每次检索返回最相似的 3 个文档片段，k 的大小可以根据需要调整，较大的 k 值会返回更多的文档片段，但可能会包含较多无关信息，也可以通过 score 的大小进行初筛。

In [31]:
query="Top-K 和 Top-P 的区别是什么？"
retrieved_docs = vectorstore.similarity_search_with_score(query=query,k=3)
j=0
for doc,score in retrieved_docs:
    j+=1
    print(f"Document {j} Score: {score}:")
    print(f"Content: {doc.page_content}\n")
print(j)

Document 1 Score: 0.8982221484184265:
Content: 输出:

Top-K 采样选择的词汇和对应的概率: 
C: 0.22
B: 0.33
A: 0.44

Top-P 采样详解

工作原理

Top-P 采样（又称 Nucleus Sampling）是一种动态选择候选词汇的方法。与 Top-K 采样不同，Top-P 采样不是固定选择 $K$ 个词汇，而是选择一组累计概率达到 $P$ 的词汇集合（即从高到低加起来的概率）。这意味着 Top-P 采样可以根据当前的概率分布动态调整候选词汇的数量，从而更好地平衡生成的多样性和质量。

步骤:

获取概率分布: 模型为每个可能的下一个词汇生成一个概率分布。

排序概率: 将词汇按照概率从高到低排序。

累积概率: 计算累积概率，直到达到预设的阈值 $P$。

筛选 Top-P: 选择累积概率达到 $P$ 的最小词汇集合。

重新归一化: 将筛选后的词汇概率重新归一化。

采样: 根据重新归一化后的概率分布，从 Top-P 词汇中随机采样一个词汇作为下一个生成的词。

数学表述

设 $V$ 为词汇表, $P(y|Y)$ 为在给定上下文 $Y$ 下生成词汇 $y$ 的概率。

Document 2 Score: 0.9301270246505737:
Content: 什么是 Top-K 和 Top-P 采样？Temperature 如何影响生成结果？

在文章 09 中我们探讨了 Beam Search 和 Greedy Search，现在来聊聊 model.generate() 中常见的三个参数: top-k, top-p 和 temperature。

代码文件下载

在线链接：Kaggle | Colab

目录

采样方法概述

Top-K 采样详解

工作原理

数学表述

代码示例

Top-P 采样详解

工作原理

数学表述

代码示例

Temperature 的作用

工作原理

代码示例

在大模型中的应用

Top-K 和 Top-P 采样是否可以一起使用？

如果我只想使用 Top-K 或者 Top-P 应该怎么办？

参考链接

在生成文本时，模型为每个可能的下一个词汇分配一个概率分布，选择下一个词汇的策略直接决定了输出的质量和多样性。以下是几种常见的选择

In [32]:
# 需要注意的是，在这里，score得分越低表示相似度越高

In [33]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # 本地
# model_path = './Breeze-7B-Instruct-v0_1'

# # 加载
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     torch_dtype="auto",  # 自动选择模型的权重数据类型
#     device_map="cuda",   # 使用gpu
# )
# 使用Openai API进行RAG模型的文本生成
from langchain_openai import ChatOpenAI
import openai
import os
key=os.environ['OPENAI_API_KEY']
base_url=os.environ['OPENAI_BASE_URL']
model=os.environ['OPENAI_MODEL']
# print(key,base_url,model)
# client=openai.OpenAI(
#     api_key=key,
#     base_url=base_url
# )
llm=ChatOpenAI(
    model_name=model,
    api_key=key,
    base_url=base_url,
)
# 文本生成
# response=client.chat.completions.create(
#     model=model,
#     messages=[
#         {
#             'role':'user',
#             'text': '你好，我是RAG模型。'
#         }
#     ],
#     max_tokens=100)
# print(response.choices[0].message.content)

In [34]:
# from transformers import pipeline

# generator = pipeline(
#     "text-generation",  # 指定任务类型为文本生成
#     model=model,
#     tokenizer=tokenizer,
#     max_length=4096,    # 指定生成文本的最大长度
#     pad_token_id=tokenizer.eos_token_id
# )

In [35]:
# from langchain_huggingface import HuggingFacePipeline
# # 使用 LangChain 的 HuggingFacePipeline 将生成器包装为 LLM 接口：
# llm = HuggingFacePipeline(pipeline=generator)

In [36]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    template="""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:""",
    input_variables=["context", "question"]
)


In [37]:
from langchain.chains import RetrievalQA
# 使用检索器和 LLM 创建问答链：
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",   # 直接堆叠所有检索到的文档
    retriever=retriever,  # 使用先前定义的检索器来获取相关文档
    # chain_type_kwargs={"prompt": custom_prompt}  # 可以选择传入自定义提示模板（传入的话记得取消注释），如果不需要可以删除这个参数
)

In [38]:
# 提出问题
query = "Top-K 和 Top-P 的区别是什么？"

# 获取答案
answer = qa_chain.invoke(query)
# print(answer)  # 可以对比 qa_chain.run() 和 qa_chain.invoke() 在返回上的差异
print(answer['result'])

Top-K 和 Top-P 是两种不同的采样方法，用于在生成文本时选择下一个词汇。它们的区别如下：

### **Top-K 采样**
- **定义**：从模型预测的词汇概率分布中，选择概率最高的前 $ K $ 个词汇作为候选，然后从这 $ K $ 个词汇中按照它们的概率分布随机采样。
- **特点**：
  - 固定选择 $ K $ 个词汇作为候选，无论这些词汇的概率分布如何。
  - 如果 $ K $ 值较小，生成的文本会更加确定性和保守；如果 $ K $ 值较大，生成的文本会更加多样。
- **优点**：实现简单，易于控制候选词汇的数量。
- **缺点**：不能动态调整候选词汇数量，可能忽略一些低概率但合理的词汇。

### **Top-P 采样（Nucleus Sampling）**
- **定义**：从模型预测的词汇概率分布中，按概率从高到低排序，累计概率直到达到一个阈值 $ P $，然后选择这些累计概率达到 $ P $ 的词汇作为候选，并从中随机采样。
- **特点**：
  - 动态选择候选词汇，根据概率分布调整候选数量。
  - 如果概率分布集中在少数几个词汇上，候选数量较少；如果概率分布较分散，候选数量较多。
- **优点**：能更灵活地平衡生成文本的多样性和质量。
- **缺点**：实现稍复杂，候选词汇数量不固定。

### **总结对比**
| 特性                | Top-K 采样                        | Top-P 采样                         |
|---------------------|----------------------------------|------------------------------------|
| **候选词汇数量**     | 固定（前 $ K $ 个词汇）         | 动态（累计概率达到 $ P $ 的词汇） |
| **灵活性**           | 较低，需手动设置 $ K $          | 较高，根据分布动态调整             |
| **生成多样性**       | $ K $ 越大，多样性越高          | $ P $ 越大，多样性越高            |
| **适用

Top-K 和 Top-P 是两种不同的采样方法，用于在生成文本时选择下一个词汇。它们的区别如下：

### **Top-K 采样**
- **定义**：从模型预测的词汇概率分布中，选择概率最高的前 $ K $ 个词汇作为候选，然后从这 $ K $ 个词汇中按照它们的概率分布随机采样。
- **特点**：
  - 固定选择 $ K $ 个词汇作为候选，无论这些词汇的概率分布如何。
  - 如果 $ K $ 值较小，生成的文本会更加确定性和保守；如果 $ K $ 值较大，生成的文本会更加多样。
- **优点**：实现简单，易于控制候选词汇的数量。
- **缺点**：不能动态调整候选词汇数量，可能忽略一些低概率但合理的词汇。

### **Top-P 采样（Nucleus Sampling）**
- **定义**：从模型预测的词汇概率分布中，按概率从高到低排序，累计概率直到达到一个阈值 $ P $，然后选择这些累计概率达到 $ P $ 的词汇作为候选，并从中随机采样。
- **特点**：
  - 动态选择候选词汇，根据概率分布调整候选数量。
  - 如果概率分布集中在少数几个词汇上，候选数量较少；如果概率分布较分散，候选数量较多。
- **优点**：能更灵活地平衡生成文本的多样性和质量。
- **缺点**：实现稍复杂，候选词汇数量不固定。

### **总结对比**
| 特性                | Top-K 采样                        | Top-P 采样                         |
|---------------------|----------------------------------|------------------------------------|
| **候选词汇数量**     | 固定（前 $ K $ 个词汇）         | 动态（累计概率达到 $ P $ 的词汇） |
| **灵活性**           | 较低，需手动设置 $ K $          | 较高，根据分布动态调整             |
| **生成多样性**       | $ K $ 越大，多样性越高          | $ P $ 越大，多样性越高            |
| **适用场景**         | 简单场景或对候选数量有明确要求   | 需要动态调整多样性和质量的场景     |

### **总结**
Top-K 是一种固定候选数量的采样方法，而 Top-P 是一种动态调整候选数量的采样方法。Top-P 通常能更好地平衡生成文本的质量和多样性。